# MATRICE DI CORRELAZIONE

Come prima operazione, importiamo le librerie necessarie

In [1]:
import pandas as pd
import statistics as st
from rpy2.robjects.packages import importr
import rpy2.robjects as ro
import numpy as np
from sklearn.preprocessing import scale
from rpy2.robjects import pandas2ri

d:\dev-cpp\anaconda\lib\site-packages\rpy2\robjects\pandas2ri.py:17: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex


Successivamente si procede con l'importazione del dataset contenuto nella cartella "data". Per raggiungere tale scopo si utilizza il metodo **read_excel()** del modulo **pandas**

In [2]:
table = pd.read_excel('https://github.com/collab-uniba/PanelJamDataExtractor/blob/master/data/TabellaCompleta.xlsx?raw=true', index = False)

Una volta importato il dataset si devono eliminare alcune feature contenute in esso, questo perchè:

-le feature **id_prog**,**id_panel**,**panel_author** contengono delle stringhe che indicano rispettivamente, l'id dei progetti estratti, l'id dei panel contenuti in ogni progetto e il nome del creatore di ogni panel. Queste feature non sono rilevanti per il calcolo della matrice di correlazione(mostrata successivamente) e nemmeno per la costruzione del modello di regressione.

-le feature **Likes**, **Views**, **Comments** e **time** fanno riferimento ai progetti e non ai singoli panel, per questo motivo non vengono considerati.

Si procede con la costruzione della matrice di correlazione con lo scopo di osservare eventuali problemi di **multicollinearità** tra le feature. Se il grado di correlazione tra due feature è maggiore del limite consentito di **0.7**, se ne deve eliminare una delle due. La matrice verrà costruita utilizzando il metodo **corr()** del modulo **pandas**

In [3]:
table = table.drop(columns = ['id_prog','id_panel','Likes','Views','Comments','panel_author','author_stars','time'])

corrMatrix = table.corr()
corrMatrix.style.background_gradient(cmap='coolwarm').set_precision(2)

,project_depth,extended,has_avatar_author,has_bio_author,author_followers,tot_loves_author,tot_views_author,author_ranking,panel_stars
project_depth,1.00,-0.42,0.06,0.03,0.02,0.04,0.05,0.07,0.07
extended,-0.42,1.00,-0.03,-0.03,-0.02,-0.03,-0.03,-0.04,-0.17
has_avatar_author,0.06,-0.03,1.00,0.24,0.18,0.15,0.11,0.09,0.08
has_bio_author,0.03,-0.03,0.24,1.00,0.22,0.12,0.01,-0.03,0.06
author_followers,0.02,-0.02,0.18,0.22,1.00,0.71,0.63,0.33,0.18
tot_loves_author,0.04,-0.03,0.15,0.12,0.71,1.00,0.76,0.31,0.19
tot_views_author,0.05,-0.03,0.11,0.01,0.63,0.76,1.00,0.63,0.12
author_ranking,0.07,-0.04,0.09,-0.03,0.33,0.31,0.63,1.00,0.12
panel_stars,0.07,-0.17,0.08,0.06,0.18,0.19,0.12,0.12,1.00


Come si può osservare dalla matrice, c'è un alto grado di correlazione tra **tot_loves_author** e **author_stars** (**0.76**) superiore al limite di **0.7**. Questo è dovuto dalla presenza di entrambe le feature nel calcolo del ranking d'autore, per questo si decide di eliminare tot_views_author. 

In [4]:
table = table.drop(columns = ['tot_views_author'])
corrMatrix = table.corr()
corrMatrix.style.background_gradient(cmap='coolwarm').set_precision(2)

,project_depth,extended,has_avatar_author,has_bio_author,author_followers,tot_loves_author,author_ranking,panel_stars
project_depth,1.00,-0.42,0.06,0.03,0.02,0.04,0.07,0.07
extended,-0.42,1.00,-0.03,-0.03,-0.02,-0.03,-0.04,-0.17
has_avatar_author,0.06,-0.03,1.00,0.24,0.18,0.15,0.09,0.08
has_bio_author,0.03,-0.03,0.24,1.00,0.22,0.12,-0.03,0.06
author_followers,0.02,-0.02,0.18,0.22,1.00,0.71,0.33,0.18
tot_loves_author,0.04,-0.03,0.15,0.12,0.71,1.00,0.31,0.19
author_ranking,0.07,-0.04,0.09,-0.03,0.33,0.31,1.00,0.12
panel_stars,0.07,-0.17,0.08,0.06,0.18,0.19,0.12,1.00


# COUNT DATA MODEL

Per costruire i count data model si uttilizza il modulo **rpy2**, il quale permette di poter importare ed utilizzare l'ambiente **R** in Python.

In [5]:
utils = importr('utils')
base = importr('base')

Installiamo tutte le librerie R necessarie. Questo può essere fatto grazie all'utilizzo del metodo **install_packages()** della libreria R **utils**.

In [ ]:
utils.install_packages('pscl')
utils.install_packages('MASS')
utils.install_packages('stats')
utils.install_packages('lmtest')
utils.install_packages('nonnest2')
utils.install_packages('AER')

Una volta installate si passa all'import delle stesse.

In [6]:
MASS = importr('MASS')
stats = importr('stats')
pscl = importr('pscl')
base = importr('base')
lmtest = importr('lmtest')
nonnest2 = importr('nonnest2')
AER = importr('AER')

Il primo passaggio, per la costruzione dei count data model, consiste nel dividere il dataset in due, dove il primo conterrà solo le feature non numeriche ed è stato etichettato con il nome **logit**, il secondo conterrà solo le feature numeriche ed è stato etichettato con il nome **numerical**. In seguito si spiegherà il motivo di questa divisione.

In [7]:
featureTable = table
#featureTable['time'] = numTime
logit = featureTable.drop(columns = ['panel_depth','author_followers','tot_loves_author',
                'author_ranking','panel_stars'])
numerical = featureTable.drop(columns = ['extended','has_avatar_author','has_bio_author'])

bisogna effettuare una log-trasformazione delle feature numeriche in quanto alcune di esse contengono valori troppo grandi. Dopo di chè si procede alla ricostruzione della tabella, andando ad unire la tabella numerica(ottenuta dalla log-normalizzazione) alla tabella non numerica. Bisogna specificare che, dopo aver effettuato la log-trasformazione, i valori della variabile dipendente **panel_stars** vengono convertiti ad intero. Questo perchè non è possibile avere una variabile dipendente che sia di un tipo diverso da tipo intero.

In [9]:
normTable = np.log1p(numerical)
normTable['panel_stars'] = normTable['panel_stars'].astype(int)
normTable['extended'] = logit['extended']
normTable['has_avatar_author'] = logit['has_avatar_author']
normTable['has_bio_author'] = logit['has_bio_author']

A questo punto si procede con la conversione del dataframe da **pandas DataFrame** ad **R DataFrame**, questa operazione è necessaria affinchè si possa utilizzare il dataFrame. Per effettuare tale conversione si usa il metodo **pandas2ri.py2ri** del modulo rpy2. 

In [10]:
pandas2ri.activate()
rTable = pandas2ri.py2ri(normTable)

lo script successivo permette la costruzione dei count data model, in questo caso si è deciso di costruire 4 modelli, ossia **Poisson**, **Negative Binomial**, **Zero Inflated negative binomial** e **Hurdle Model**, utilizzando rispettivamente la funzione **glm** del package **stats** per il modello di Poisson, la funzione **glm_nb** del package **MASS** per la Negative Binomial, la funzione **zeroinfl** del package **pscl** per la Zero Inflated è la funzione **hurdle** del package **pscl** per Hurdle Model. La variabile **formula** contiene una stringa scritta in notazione **Patsy** utile per poter impostare la variabile dipendente (inserita a sinistra del simbolo tilde ~ ) e le variabili indipendenti che fungeranno da predittori(poste a destra del simbolo tilde). 

In [11]:
formula = """panel_stars ~ panel_depth + author_followers +
             author_ranking + extended + has_avatar_author + has_bio_author + tot_loves_author"""


summary = ro.r['summary']

poissonMod = stats.glm(ro.r(formula), family = ro.r('poisson(link = "log")'), data = rTable)
nbMod = MASS.glm_nb(ro.r(formula), data = rTable)
zeroinflMod = pscl.zeroinfl(ro.r(formula), data = rTable,dist = 'negbin')
hurdleMod = pscl.hurdle(ro.r(formula), data = rTable, dist = 'negbin', zero = "binomial")

# RISULTATI COUNT DATA MODEL

**MODELLO DI POISSON**

In [48]:
poissonSum = summary(poissonMod)
loglikPoisson = stats.logLik(poissonMod)

poisson = str(poissonSum)
poisson = poisson.split('Deviance')[1]
print(poisson)
print(loglikPoisson)

 Residuals: 

    Min       1Q   Median       3Q      Max  

-1.4258  -0.8871  -0.6865   0.6245   2.9196  



Coefficients:

                      Estimate Std. Error z value Pr(>|z|)    

(Intercept)           -2.38625    0.13929 -17.132  < 2e-16 ***

project_depth         -0.04333    0.03886  -1.115   0.2648    

author_followers       0.08520    0.01753   4.861 1.17e-06 ***

author_ranking         0.16503    0.01489  11.084  < 2e-16 ***

extendedTRUE          -0.45964    0.03443 -13.350  < 2e-16 ***

has_avatar_authorTRUE  0.15773    0.10118   1.559   0.1190    

has_bio_authorTRUE     0.07730    0.03243   2.383   0.0172 *  

tot_loves_author       0.08339    0.01362   6.123 9.16e-10 ***

---

Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1



(Dispersion parameter for poisson family taken to be 1)



    Null deviance: 11437  on 13747  degrees of freedom

Residual deviance: 10479  on 13740  degrees of freedom

AIC: 20749



Number of Fisher Scoring iterations: 5



'

In [44]:
print(stats.AIC(poissonMod))

[1] 20748.71



**NEGATIVE BINOMIAL**

In [47]:
nbSum = summary(nbMod)
loglikNB = stats.logLik(nbMod)

nb = str(nbSum)
nb = nb.split('Deviance')[1]
print(nb)
print(loglikNB)

 Residuals: 

    Min       1Q   Median       3Q      Max  

-1.4258  -0.8871  -0.6865   0.6245   2.9195  



Coefficients:

                      Estimate Std. Error z value Pr(>|z|)    

(Intercept)           -2.38627    0.13930 -17.130  < 2e-16 ***

project_depth         -0.04333    0.03886  -1.115   0.2648    

author_followers       0.08520    0.01753   4.861 1.17e-06 ***

author_ranking         0.16503    0.01489  11.084  < 2e-16 ***

extendedTRUE          -0.45964    0.03443 -13.350  < 2e-16 ***

has_avatar_authorTRUE  0.15772    0.10120   1.559   0.1191    

has_bio_authorTRUE     0.07730    0.03244   2.383   0.0172 *  

tot_loves_author       0.08339    0.01362   6.123 9.20e-10 ***

---

Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1



(Dispersion parameter for Negative Binomial(8237.561) family taken to be 1)



    Null deviance: 11437  on 13747  degrees of freedom

Residual deviance: 10479  on 13740  degrees of freedom

AIC: 20751



Number of Fisher Scorin

In [46]:
print(stats.AIC(nbMod))

[1] 20750.9



**ZERO INFLATED NEGATIVE BINOMIAL**

In [13]:
zeroinflSum = summary(zeroinflMod)
loglikZeroinfl = stats.logLik(zeroinflMod)

zeroinfl = str(zeroinflSum)
zeroinfl = zeroinfl.split('Pearson')[1]
print(zeroinfl)
print(loglikZeroinfl)

 residuals:

    Min      1Q  Median      3Q     Max 

-0.9456 -0.6378 -0.4617  0.6907  9.3265 



Count model coefficients (negbin with log link):

                      Estimate Std. Error z value Pr(>|z|)    

(Intercept)           -1.91475    0.18944 -10.107  < 2e-16 ***

project_depth         -0.06293    0.04158  -1.513    0.130    

author_followers       0.02283    0.01981   1.152    0.249    

author_ranking         0.10546    0.01741   6.056 1.39e-09 ***

extendedTRUE          -0.41006    0.03702 -11.076  < 2e-16 ***

has_avatar_authorTRUE  0.08566    0.14771   0.580    0.562    

has_bio_authorTRUE     0.05618    0.03472   1.618    0.106    

tot_loves_author       0.11726    0.01487   7.885 3.14e-15 ***

Log(theta)            16.21801         NA      NA       NA    



Zero-inflation model coefficients (binomial with logit link):

                      Estimate Std. Error z value Pr(>|z|)    

(Intercept)             5.3310     1.4921   3.573 0.000353 ***

project_depth     

In [53]:
print(stats.AIC(zeroinflMod))

[1] 20672.64



**HURDLE MODEL**

In [14]:
hurdleSum = summary(hurdleMod)
loglikHurdle = stats.logLik(hurdleMod)

hurdle = str(hurdleSum)
hurdle = hurdle.split('Pearson')[1]
print(hurdle)
print(loglikHurdle)

 residuals:

    Min      1Q  Median      3Q     Max 

-1.4263 -0.7262 -0.4911  0.8451  7.6005 



Count model coefficients (truncated negbin with log link):

                      Estimate Std. Error z value Pr(>|z|)    

(Intercept)           -1.40920    0.55982  -2.517   0.0118 *  

project_depth         -0.11960    0.13911  -0.860   0.3899    

author_followers       0.27035    0.06241   4.332 1.48e-05 ***

author_ranking        -0.03220    0.05219  -0.617   0.5373    

extendedTRUE          -0.57208    0.11384  -5.025 5.03e-07 ***

has_avatar_authorTRUE  0.49586    0.45762   1.084   0.2786    

has_bio_authorTRUE    -0.18502    0.10559  -1.752   0.0797 .  

tot_loves_author      -0.09076    0.04729  -1.919   0.0549 .  

Log(theta)            13.68812         NA      NA       NA    

Zero hurdle model coefficients (binomial with logit link):

                      Estimate Std. Error z value Pr(>|z|)    

(Intercept)           -2.79278    0.17494 -15.964  < 2e-16 ***

project_depth

In [52]:
print(stats.AIC(hurdleMod))

[1] 19946.39



Prima di determinare quale dei 4 modelli è il migliore, bisogna fare una precisazione tra il modello di Poisson e la Negative Binomial. Dopo aver modellato i dati, bisogna verificare se essi presentano un fenomeno di **sovradispersione**, **equidispersione** o **sottodispersione**, nello specifico si ha:

- Un fenomeno di **sovradispersione** se la varianza dei dati è maggiore della loro media.
- Un fenomeno di **equidispersione** se la varianza dei  dati è uguale della loro media.
- Un fenomeno di **sottodispersione** se la varianza dei dati è minore dellla loro media.

E' importante fare queste distinzioni in quanto il modello di Poisson assume che la varianza sia uguale alla media(quindi quando si ha equidispersione), mentre la Negative Ninomial è un modello che si basa sulla Poisson ma che viene utilizzato qualora si dovesse avere sovradispersione. Se invece ci dovesse essere sottodispersione, entrambi i modelli non sono i più adatti per poter modellare i dati e si dovrà ricorrere all'utilizzo di altri modelli. Prima di arrivare a ciò, però, si deve valutare quale dei due modelli sia il migliore per modellare i dati. Per raggiungere tale scopo si effettua un test detto **"dispersion test"** il quale effettua delle ipotesi e verifica la correlazione che sussiste tra la varianza e la media. E' possibile effettuare tale test con la funzione **dispersiontest** della libreria R **AER**; in particolare si andrà ad osservare un coefficiente **"alpha"**, se esso ha un valore minore di 0, allora si ha sottodispersione, se invece dovesse avere un valore pari a 0 si ha equidispersione, altrimenti si ha sovradispersione.

In [41]:
c = ro.r['c']
dispTest = AER.dispersiontest(poissonMod,trafo = 1,alternative = c("greater", "two.sided", "less"))
print(dispTest.rx(3))

$estimate

     alpha 

-0.2363538 





come si evince dall'output della funzione, il valore di alpha è pari a **-0.236**, questo implica che, seppure di poco, si ha sottodispersione dei dati. Questo spiega il perchè la Poisson e la Negative Binomial hanno dei valori di AIC quasi identici, infatti la prima ha un AIC di **20748**, la seconda invece ha un AIC di **20750**. Osservando anche i valori di **Likelihood** di entrambi i modelli(**-10366**) si arriva alla conclusione che essi sono identici e non possono essere attendibili. Come metodo di paragone per gli altri due modelli costruiti(Zero Inflated Negative Binomial e Hurdle Model) si decide di utilizzare la Poisson in quanto l'AIC è, seppur di poco, minore dell'AIC della Negative Binomial.

**TEST DI VUONG**

Per verificare quale dei seguenti modelli è il migliore per modellare i dati, si utilizza il test di Vuong.

In [49]:
pm_zeroinfl_vuong = nonnest2.vuongtest(zeroinflMod,poissonMod)
print(pm_zeroinfl_vuong)



Model 1 

 Class: zeroinfl 

 Call: (function (formula, data, subset, na.action, weights, offset, ...



Model 2 

 Class: glm 

 Call: (function (formula, family = gaussian, data, weights, subset, ...



Variance test 

  H0: Model 1 and Model 2 are indistinguishable 

  H1: Model 1 and Model 2 are distinguishable 

    w2 = 0.006,   p = 9.81e-05



Non-nested likelihood ratio test 

  H0: Model fits are equal for the focal population 

  H1A: Model 1 fits better than Model 2 

    z = 5.173,   p = 1.15e-07

  H1B: Model 2 fits better than Model 1 

    z = 5.173,   p = 1



Il primo test effettuato è tra la Zero Inflated Negative Binomial e la Poisson. La Zero Inflated è un modello utilizzato per dati che esibiscono un fenomeno di sovradispersione e che contengono un eccesso di zeri. Essa combina la distribuzione della Negative Binomial con la distribuzione **logit**. Come si evince sia dal Test di vuong, sia dall'AIC della Zero Inflated(**20672**<**20748**), quest'ultimo è migliore rispetto alla Poisson. E' opportuno precisare che lo scarto è molto basso.

In [50]:
pm_hurdle_vuong = nonnest2.vuongtest(hurdleMod,poissonMod)
print(pm_hurdle_vuong)



Model 1 

 Class: hurdle 

 Call: (function (formula, data, subset, na.action, weights, offset, ...



Model 2 

 Class: glm 

 Call: (function (formula, family = gaussian, data, weights, subset, ...



Variance test 

  H0: Model 1 and Model 2 are indistinguishable 

  H1: Model 1 and Model 2 are distinguishable 

    w2 = 0.048,   p = 1.74e-08



Non-nested likelihood ratio test 

  H0: Model fits are equal for the focal population 

  H1A: Model 1 fits better than Model 2 

    z = 15.915,   p = <2e-16

  H1B: Model 2 fits better than Model 1 

    z = 15.915,   p = 1



Il valore di AIC dell'Hurdle Model è pari a **19946** il quale è nettamente inferiore rispetto all'AIC della Poisson. Sia questo, sia i risultati ottenuti dal test di Vuong, dimostrano che l'Hurdle Model è migliore rispetto al modello di Poisson.

In [51]:
zero_hurdle_test = nonnest2.vuongtest(zeroinflMod, hurdleMod)
print(zero_hurdle_test)



Model 1 

 Class: zeroinfl 

 Call: (function (formula, data, subset, na.action, weights, offset, ...



Model 2 

 Class: hurdle 

 Call: (function (formula, data, subset, na.action, weights, offset, ...



Variance test 

  H0: Model 1 and Model 2 are indistinguishable 

  H1: Model 1 and Model 2 are distinguishable 

    w2 = 0.055,   p = <2e-16



Non-nested likelihood ratio test 

  H0: Model fits are equal for the focal population 

  H1A: Model 1 fits better than Model 2 

    z = -13.180,   p = 1

  H1B: Model 2 fits better than Model 1 

    z = -13.180,   p = < 2.2e-16



L'ultimo test che si deve effettuare è tra i due modelli migliori, Hurdle Model  e Zero Inflated. Innanzitutto, il confronto dei rispettivi AIC mostra come il valore del primo è minore del secondo (**19946**<**20672**), questo fattore unito ai risultati ottenuti dal test di Vuong dimostrano come l'Hurdle Model sia migliore rispetto alla Zero Inflated Negative Binomial.

# CALCOLO INTERVALLO DI CONFIDENZA HURDLE MODEL

In [16]:
conf_int = stats.confint(hurdleMod)
row = 1
_1_count_conf_int = []
_2_count_conf_int = []
while row <= (conf_int.nrow/2):
    _1_count_conf_int.append((conf_int.rx(row,1))[0])
    _2_count_conf_int.append((conf_int.rx(row,2))[0])
    row = row + 1
    
row = 9
_1_zero_conf_int = []
_2_zero_conf_int = []
while row <= (conf_int.nrow):
    _1_zero_conf_int.append((conf_int.rx(row,1))[0])
    _2_zero_conf_int.append((conf_int.rx(row,2))[0])
    row = row + 1

# CALCOLO STANDARDIZED FACTOR CHANGE COEFFICIENTI HURDLE MODEL COUNT 

In [14]:
value = base.matrix(base.exp(stats.coef(hurdleMod, "count")))

data = {'features': ['Intercept','panel_depth','author_followers','author_ranking','extended_TRUE',
                     'has_avatar_author_TRUE','has_bio_author_TRUE','tot_loves_author'],
        'SFC': value,
        '0,025': _1_count_conf_int,
        '0,975': _2_count_conf_int}
SFC = pd.DataFrame(data,columns = ['features','SFC','0,025','0,975'])
print(SFC)

                 features       SFC     0,025     0,975
0               Intercept  0.244339 -2.506425 -0.311974
1           project_depth  0.887272 -0.392261  0.153054
2        author_followers  1.310428  0.148026  0.392682
3          author_ranking  0.968313 -0.134492  0.070093
4           extended_TRUE  0.564348 -0.795208 -0.348959
5  has_avatar_author_TRUE  1.641914 -0.401061  1.392786
6     has_bio_author_TRUE  0.831084 -0.391969  0.021921
7        tot_loves_author  0.913238 -0.183438  0.001919


# CALCOLO ODDS RATIO COEFFICIENTI HURDLE MODEL ZERO

## Che cos'è l'odds ratio

l'**Odds Ratio**(**OR**) è un modo per rappresentare la forza dell'associazione tra fattori di esposizione (nel nostro caso i predittori) e l'esito o risultato(ovvero la variabile dipendente). Possiamo distinguere 3 casi di OR:

- Un OR > 1 significa maggiore probabilità di associazione tra l'esposizione e il risultato. Nello specifico, un OR di 1.2 significa che c'è un aumento del 20% delle probabilità di un risultato con una data esposizione. In questo caso significa che se un predittore dovesse avere un OR di 1.2, si avrebbe un aumento del 20% della probabilità che un panel riceva una star(dato che la variabile dipendente è panel_star e stiamo considerando la parte logit del modello) con quel predittore.

- Un OR < 1 significa minore probabilità di associazione tra l'esposizione e il risultato. Nello specifico, un OR di 0.2 significa che c'è una diminuzione dell' 80% delle probabilità di un risultato con una data esposizione. In questo caso significa che se un predittore dovesse avere un OR di 0.2, si avrebbe una diminuzione dell' 80% della probabilità che un panel riceva una star con quel predittore.

- Un OR = 1 significa che non c'è nessuna associazione tra l'esposizione e il risultato.

**ATTENZIONE!**
Un OR pari a 0.2 non implica che c'è un aumento del 20% di probabilità che un panel riceva una star con un determinato predittore, stesso discorso vale per un OR pari a 1.2, con ciò non significa che c'è una diminuzione dell' 80% che un panel non riceva una star. Un predittore con OR < 1 influisce **solo** negativamente sul risultato, mentre un  OR > 1 influisce **solo** positivamente. 

In [17]:
values = base.matrix((base.exp(stats.coef(hurdleMod, "zero"))))
_1_zero_conf_int = np.exp(_1_zero_conf_int)
_2_zero_conf_int = np.exp(_2_zero_conf_int)
data = {'features': ['Intercept','panel_depth','author_followers','author_ranking','extended_TRUE',
                     'has_avatar_author_TRUE','has_bio_author_TRUE','tot_loves_author'],
        'Odds Ratio': values,
        '0,025': _1_zero_conf_int,
        '0,975': _2_zero_conf_int}
oddsRatioTable = pd.DataFrame(data,columns = ['features','Odds Ratio','0,025','0,975'])
print(oddsRatioTable)

                 features  Odds Ratio     0,025     0,975
0               Intercept    0.061251  0.043471  0.086302
1           project_depth    0.944951  0.853817  1.045813
2        author_followers    1.129810  1.080111  1.181795
3          author_ranking    1.332923  1.281033  1.386915
4           extended_TRUE    0.475429  0.431198  0.524197
5  has_avatar_author_TRUE    1.083566  0.859714  1.365705
6     has_bio_author_TRUE    1.164283  1.066918  1.270532
7        tot_loves_author    1.138848  1.099865  1.179213


Prima di analizzare i risultati ottenuti bisogna specificare che cos'è un intervallo di confidenza e perchè è importante.
Quando si effettua la stima di un parametro, in questo caso il coefficiente di un predittore, è opportuno accompagnare tale stima da un range di valori plausibili per quel parametro, il quale è appunto l'**intervallo di confidenza**. Ad ogni intervallo di confidenza viene associato un livello di confidenza che rappresenta il grado di attendibilità dell'intervallo. In questo caso è stato impostato un grado di confidenza del 95%, esso indica la percentuale con la quale il valore reale del parametro è compreso tra l'estremo inferiore e l'estremo superiore dell'intervallo. Dopo aver calcolato tutti i valori di OR, si deve anche calcolare l'intervallo di confidenza al 95% di questi ultimi, questo perchè, se nell'intervallo di confidenza di un OR è compreso il valore 1, allora il predittore a cui fa riferimento l'OR, è considerato **statisticamente non significativo**. Alla luce di queste considerazioni si può affermare che:

- La variabile **panel_depth** risulta essere statisticamente non significativa in quanto contiene il valore 1 nell'intervallo **[0.854,1.045]**.
- La variabile **author_followers** risulta essere statisticamente significativa ed ha un OR pari a **1.13**. Questo implica che esso è un predittore influente sulla variabile dipendente, infatti aumenta del **~13%** la probabilità che un panel riceva una star.
- La variabile **author_ranking** risulta essere statisticamente significativa ed ha un OR pari a **1.333**. Questo implica che esso è un predittore influente sulla variabile dipendente, infatti aumenta del **~33%** la probabilità che un panel riceva una star.
- La variabile **extended_TRUE** risulta essere statisticamente significativa ed ha un OR pari a **0.475**. Questo implica che esso è un predittore che influisce molto negativamente sulla variabile dipendente, infatti dimiunisce del **~53%** la probabilità che un panel riceva una star.
- La variabile **has_avatar_author_TRUE** risulta essere statisticamente non significativa in quanto contiene il valore 1 nell'intervallo **[0.860,1.366]**.
- La variabile **har_bio_author_TRUE** risulta essere statisticamente significativa ed ha un OR pari a **1.164**. Questo implica che esso è un predittore che influisce sulla variabile dipendente, infatti aumenta del **~16%** la probabilità che un panel riceva una star.
- La variabile **tot_loves_author** risulta essere statisticamente significativa ed ha un OR pari a **1.139**. Questo implica che esso è un predittore che influisce sulla variabile dipendente, infatti aumenta del **~14%** la probabilità che un panel riceva una star.